![](http://sigdelta.com/assets/images/sages-sd-logo.png)

# Analiza danych i uczenie maszynowe w Python

Autor notebooka: Jakub Nowacki.

## Dane tekstowe

W tym notebooku prezentujemy podejście do analizy, wektoryzacji danych i uczenia maszynowego z użyciem scikit-learn i innych pakietów w Python. Kroki te są wstępem do uczenia maszynowego.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import glob
import matplotlib as mpl

# Parametry wykresów
mpl.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['font.size'] = 12

## Macierz Term-Document

[Macierz Term-Document (TD)](https://en.wikipedia.org/wiki/Document-term_matrix), jest to macierz, w której kolumny stanowią unikalne słowa, a wiersze stanowią wartości ile razy dane słowo wystąpiło w dokumencie. Dla przykładu (za Wikipedią), jeżeli mamy dwa dokumenty:

* D1 = "I like databases"
* D2 = "I hate databases"

to macierz TD możemy przedstawić jako

|    | I | like | hate | databases |
|----|---|------|------|-----------|
| **D1** | 1 | 1    | 0    | 1         |
| **D2** | 1 | 0    | 1    | 1         |

Poniżej ten sam przykład zrealizowany w Pandas:

In [2]:
sample = pd.DataFrame({
    'docs': ['D1', 'D2'],
    'lines': ['I like databases Databases', 'I hate databases']
})
sample

,docs,lines
0,D1,I like databases Databases
1,D2,I hate databases


In [3]:
sample['words'] = sample.lines.str.strip().str.lower().str.split('[\W_]+')
sample

,docs,lines,words
0,D1,I like databases Databases,"[i, like, databases, databases]"
1,D2,I hate databases,"[i, hate, databases]"


In [4]:
rows = list()
for row in sample[['docs', 'words']].iterrows():
    r = row[1]
    for word in r.words:
        rows.append((r.docs, word))

words = pd.DataFrame(rows, columns=['docs', 'word'])
words

,docs,word
0,D1,i
1,D1,like
2,D1,databases
3,D1,databases
4,D2,i
5,D2,hate
6,D2,databases


In [5]:
words.pivot_table(index='docs', 
                  columns='word', 
                  aggfunc=lambda v: v['word'].count())\
    .fillna(0)

word,databases,hate,i,like
docs,,,,
D1,2.0,0.0,1.0,1.0
D2,1.0,1.0,1.0,0.0


### Zadanie

1. Używając Pandas obliczyć macierz TD dla książek.
1. Co może wpłynąć na wynik?

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
count_vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
sample['lines']

0    I like databases Databases
1              I hate databases
Name: lines, dtype: object

In [8]:
X = count_vectorizer.fit_transform(sample['lines'])
X

<2x3 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [9]:
print(X)

  (0, 0)	2
  (0, 2)	1
  (1, 1)	1
  (1, 0)	1


In [10]:
X.toarray()

array([[2, 0, 1],
       [1, 1, 0]], dtype=int64)

In [11]:
count_vectorizer.get_feature_names()

['databases', 'hate', 'like']

In [12]:
pd.DataFrame(X.toarray(), 
             columns=count_vectorizer.get_feature_names(), 
             index=sample['docs'])

,databases,hate,like
docs,,,
D1,2,0,1
D2,1,1,0


## TF-IDF

Kolejną techniką jest poznane uprzednio [Term Frequency–Inverse Document Frequency (TF-IDF)](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). Jest on popularnym algorytmem do analizy danych tekstowych, używany dość często w pozyskiwaniu danych (data mining).

Poniżej przykładowe obliczenie TF-IDF z użyciem wektoryzera scikit-learn.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [14]:
X = tfidf_vectorizer.fit_transform(sample['lines'])
X

<2x3 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [15]:
X.toarray()

array([[0.81818021, 0.        , 0.57496187],
       [0.57973867, 0.81480247, 0.        ]])

In [16]:
tfidf_vectorizer.get_feature_names()

['databases', 'hate', 'like']

In [17]:
pd.DataFrame(X.toarray(), 
            columns=tfidf_vectorizer.get_feature_names(), 
            index=sample['docs'])

,databases,hate,like
docs,,,
D1,0.818180,0.000000,0.574962
D2,0.579739,0.814802,0.000000


## Miary podobieństwa

Jest wiele miar podobieństwa, które liczą jak odległe są pary wartości od siebie. Miary mogą być używane zarówno do wektorów oraz macierzy, jak i słów oraz dokumentów; zobacz [ciekawy opis](http://dataconomy.com/2015/04/implementing-the-five-most-popular-similarity-measures-in-python/) miar wraz z implementacją w Pythonie. 

Pierwszą miarą, która jest standardową miarą podobieństwa dwóch stringów jest [miara Levenshteina](https://en.wikipedia.org/wiki/Levenshtein_distance). Jest wiele implementacji używania tej miary, niemniej, najłatwiej dostępną implementacją jest zawarta w [difflib](https://docs.python.org/3/library/difflib.html), która jest częścią standardowej biblioteki Pythona.

In [ ]:
import difflib 

def similarity(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

similarity('cat', 'cats')

In [ ]:
similarity('cat', 'dog')

In [ ]:
sample

In [ ]:
similarity(sample.at[0, 'lines'], sample.at[1, 'lines'])

In [ ]:
similarity('cat', 'catepillar')

W przypadku dokumentów tekstowych, znane są przynajmniej dwie popularne miary:

* cosinusowa
* Jaccarda

**Miaria cosinusowa** zdefiniowana jest jako kąt między dwoma wektorami:

$$
sim(A, B) = \cos(\Theta) = \frac{A \cdot B}{\Vert A \Vert \cdot \Vert B \Vert}
$$

Zatem miara wymaga formy zwektowyzowanej do obliczenia wartości; musimy się najpierw posłużyć którymś wektoryzatorem aby otrzymać macierz a potem policzyć miarę.

In [ ]:
X = count_vectorizer.fit_transform(sample['lines'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(X)

**Miara Jaccarda** operuje z kolei na zbiorach i zdefiniowana jest jako:

$$
sim(A, B) = \frac{|A \cap B|}{| A \cup B |}
$$

Zatem dla naszego przykładu wygląda to następująco:


In [ ]:
A = sample.words[0]
B = sample.words[1]
A, B

In [ ]:
def sim_jaccard(A, B):
    a = set(A)
    b = set(B)
    i = set.intersection(a, b)
    u = set.union(a, b)
    print(i, u)
    return len(i)/len(u)

sim_jaccard(A, B)

In [ ]:
from sklearn.metrics import jaccard_similarity_score

list(set(A))
jaccard_similarity_score(list(set(A)), list(set(B)))

## Transformatory i pipeliny

Scikit-learn wprowadził szereg ułatwień do przetwarzania danych i tworzenia modeli. W ogólności, możemy korzystać z 3 podstawowych elementów:

* [transformer](http://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html)
* [estimator](http://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html)
* [pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline)

Transformetry (funkcje zmieniające dane) i estymatory (modele do wyuczenia) łączy się w pipeliny; więcej o tym można przeczytać [w dokumentacji](http://scikit-learn.org/stable/modules/pipeline.html).


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer

In [ ]:
td = CountVectorizer().fit_transform(sample['lines'])
tfidf_transformer.fit_transform(td)

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
tfidf_pipeline = make_pipeline(CountVectorizer(), TfidfTransformer())
tfidf_pipeline

In [ ]:
X = tfidf_pipeline.fit_transform(sample['lines'])
X.toarray()

In [ ]:
pd.DataFrame(X.toarray(), 
             index=sample['docs'], 
             columns=tfidf_pipeline.steps[0][1].get_feature_names())

Można też samemu nazywać elementy pipelinu:

In [ ]:
list(steps.items())

In [ ]:
steps = {
    'count_vect': CountVectorizer(),
    'tfidf_trans': TfidfTransformer()
}

# Pipeline oczekuje kroków jako listy z krotkami (nazwa, obiekt)
tfidf_pipeline = Pipeline(list(steps.items()))
tfidf_pipeline

In [ ]:
tfidf_pipeline.steps

In [ ]:
tfidf_pipeline.fit_transform(sample['lines'])

In [ ]:
# Co tu się dzieje?
steps['count_vect'].get_feature_names()

### Zadanie

1. Wykorzystaj narzędzia scikit-learn do stworzenie pełnego pipelinu wykonującego wektoryzację z TF-IDF dla danych z książek.


### Funkcje użytkownika

Niekiedy mamy potrzebę korzystania z funkcji wybiegających poza zestaw dostępny w scikit-learn. Są dwie metody:

* [FunctionTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html#sklearn.preprocessing.FunctionTransformer)
* Klasa dziedzicząca po [BaseEstimator i TransformerMixin](http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html#sphx-glr-auto-examples-hetero-feature-union-py)

Obecnie najpierw zalecany jest FunctionTransformer.

In [ ]:
import re
import numpy as np

@np.vectorize
def replace_database(linia):
    return re.sub('database', 'DB', linia, flags=re.IGNORECASE)

replace_database(sample['lines'])

In [ ]:
from sklearn.preprocessing import FunctionTransformer

replace_func = FunctionTransformer(replace_database, validate=False)
replace_func.fit_transform(sample['lines'])

In [ ]:
new_pipeline = make_pipeline(replace_func, TfidfVectorizer())
new_pipeline

In [ ]:
X = new_pipeline.fit_transform(sample['lines'])
X

In [ ]:
pd.DataFrame(X.toarray(), 
             index=sample['docs'], 
             columns=new_pipeline.steps[1][1].get_feature_names())

Drugą metodą tworzenia funkcji użytkownika jest stworzenie klasy, która dziedziczy po klasach `BaseEstimator` i `TransformerMixin`, czyli de facto jak to jest robione dla innych transformerów.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class ReplaceDatabaseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, replace_from='database', replace_to='DB'):
        self.replace_from = replace_from
        self.replace_to = replace_to

    def fit(self, x, y=None):
        return self

    def _replace_str(self, line):
        return re.sub(self.replace_from, 
                      self.replace_to, 
                      line, 
                      flags=re.IGNORECASE)
    
    def transform(self, data):
        func = np.vectorize(lambda line: self._replace_str(line))
        return func(data)

In [ ]:
new_pipeline2 = make_pipeline(ReplaceDatabaseTransformer(), TfidfVectorizer())
new_pipeline2

In [ ]:
X = new_pipeline2.fit_transform(sample['lines'])
X

In [ ]:
pd.DataFrame(X.toarray(), 
             index=sample['docs'], 
             columns=new_pipeline2.steps[1][1].get_feature_names())

### Zadanie

1. Napisz funkcję zamieniającą wszystkie napotkane [URLe](https://pl.wikipedia.org/wiki/Uniform_Resource_Locator) na stałą wartość, która może być podawana jako argument.

## Klasyfikacja danych tekstowych

W tej części notebooka przeprowadzimy klasyfikację danych tekstowych. Do analiz będziemy wykorzystywać kolekcję wiadomości SMS, zawierających spam i prawdziwe wiadomości, dostępnym w [repozytorium UCI](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection). Poniższy kod pobiera dane.

In [ ]:
import os
import urllib.request
import zipfile

data_path = 'data'

os.makedirs(data_path, exist_ok=True)

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
file_name = url.split('/')[-1]
dest_file = os.path.join(data_path, file_name) 

data_file = 'SMSSpamCollection'
data_full = os.path.join(data_path, data_file)

urllib.request.urlretrieve(url, dest_file)

with zipfile.ZipFile(dest_file) as zip_file:
    zip_file.extract(data_file, path=data_path)

In [ ]:
import pandas as pd

sms = pd.read_csv(data_full, 
                  sep='\t', 
                  names=['is_spam', 'text'])
sms.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_sms, test_sms = train_test_split(sms, test_size=0.2)

print('Train set:')
print(train_sms.describe())
print()
print('Test set:')
print(test_sms.describe())

Naszym zadaniem jest wytrenowanie klasyfikatora, który klasyfikuje wiadomość SMS jako spam. Podobne zadanie z dość szerokim opisem można znaleźć w [tym wpisie](https://radimrehurek.com/data_science_python/). 


### Naive Bayes

Zaczniemy od klasyfikatora [Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html). Używa on [twierdzenia Bayesa](https://en.wikipedia.org/wiki/Bayes%27_theorem) do obliczenia prawdopodobieństw poszczególnych klas w zależności od dostępnych opcji, przy założeniu niezależności zmiennych losowych (stąd naiwny). Rozpatrzmy przykład ([źródło](https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/)):

![](https://www.analyticsvidhya.com/wp-content/uploads/2015/08/Bayes_41-850x310.png)

Poniżej przykład obliczenia prawdopodobieństwa na podstawie metody Bayesa.

$$P(Yes | Sunny) = \frac{P( Sunny | Yes)  P(Yes)}{P (Sunny)}$$
$$P (Sunny |Yes) = 3/9 = 0.33$$
$$P(Sunny) = 5/14 = 0.36$$ 
$$P( Yes)= 9/14 = 0.64$$
$$P (Yes | Sunny) = \frac{0.33 \cdot 0.64}{0.36} = 0.60$$

Poniżej, przykład modelu detekcji spamu z użuciem metody Naive Bayes. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer

In [ ]:
X = vectorizer.fit_transform(train_sms['text'])
X

In [ ]:
from sklearn.naive_bayes import MultinomialNB

spam_detector = MultinomialNB().fit(X, train_sms['is_spam'])
spam_detector

In [ ]:
i = 17
train_sms.iloc[i, 0], spam_detector.predict(X[i])[0], train_sms.iloc[i, 1]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

X = vectorizer.transform(test_sms['text'])

y_pred = spam_detector.predict(X)
y_true = test_sms['is_spam']

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

### Zadanie

1. Zbuduj pipeline wektoryzatora i modelu.
1. Użyj `GridSearchCV` do znalezienia najlepszego modelu.
1. Popraw tokenizację w celu poprawienia jakości klasyfikacji.
1. Użyj innych klasyfikatorów.

## Modelowanie tematów

Modelowanie tematów (topic modelling) jest to zadanie ekstrakcji ważnych elementów z tekstu, które są jego tematami. Wiodącą biblioteką w pythonie do tego celu jest [gensim](https://radimrehurek.com/gensim/). 

### Latent Dirichlet Allocation

Jest wiele ciekawych algorytmów wykonujących to zadanie, jednak jednym z najgłośniejszych algorytmów do ekstrakcji tematów jest [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation). 

![](https://cdn-images-1.medium.com/max/1600/0*II7wZlKViCt4ssBm.png)

Jedną z najskuteczniejszych implementaci LDA w Pythonie jest implementacja dostępna w gensim, która posiada też dość rozbudowaną [dokumentację](https://radimrehurek.com/gensim/models/ldamodel.html). Wykorzystajmy zatem LDA do naszego zadania.

Do analizy wykorzystamy zbiór [20 newsgroups z scikit-learn](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html).

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(shuffle=True, random_state=1,
                                remove=('headers', 'footers', 'quotes'))

In [ ]:
newsgroups.data[:2]

In [ ]:
newsgroups.target

In [ ]:
newsgroups.target_names

Zawęzimy trochę próbkę, żeby uczenie było szybsze.

In [ ]:
n_samples = 2000
newsgroups_samples = newsgroups.data[:n_samples]

Najpierw tworzymy korpus dla LDA używając narzędzi gensim.

In [ ]:
import nltk
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel

words = [nltk.regexp_tokenize(d.lower(), '\w+') for d in newsgroups_samples]

dictionary = corpora.Dictionary(words)
corpus = [dictionary.doc2bow(text) for text in words]

Słownik posiada ID słowa jako klucz i słowo jako wartość.

In [ ]:
from itertools import islice

for k, v in islice(dictionary.items(), 10):
    print('{}: {}'.format(k, v)) 

Korpus posiada listę krotek, lista na dokument; krotki zawierają pary ID słowa i jego liczność w danych dokumencie. 

In [ ]:
print(corpus[:2])

### Zadanie

1. Dla 10 pierwszych dokumentów wypisz słowa i ich liczność.

Teraz wytrenujmy sam model LDA.

In [ ]:
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, alpha="auto")
model

In [ ]:
for i in range(10):
    print(model.get_document_topics(corpus[i], minimum_probability=0.1))

In [ ]:
k = 0
model.get_topic_terms(topicid=k)

In [ ]:
model.print_topics()

### Zadanie

1. Zmień ilość tematów; czy coś się zmieniło?
1. Zobacz jak tematy mapują się na klasy.
1. Popraw tokenizację; możesz np.:
    - usunąć nie-słowa
    - usunąć wyrazy ze stop listy
    - sprowadzić słowa do wspólnej wielkości znaku
1. Sprawdź inne metody budowania korpusu, np.:
    - macierz TD
    - TF-IDF


In [ ]:
from gensim.sklearn_api.ldamodel import LdaTransformer
from gensim.sklearn_api.text2bow import Text2BowTransformer
from sklearn.pipeline import Pipeline

steps = {
    'text2bow': Text2BowTransformer(),
    'lda': LdaTransformer(num_topics=10)
}

p = Pipeline(list(steps.items()))
p

In [ ]:
p.fit_transform(newsgroups_samples)

In [ ]:
steps['lda'].gensim_model.print_topics()

In [ ]:
steps['text2bow'].gensim_model

In [ ]:
p.transform(['ala ma kota', 'lala'])

Możemy też zastosować wbudowaną metodę scikit-learn; zobacz [dokumentację](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html).

In [ ]:
# Funkcja użytkowa do wyświetlania tematów.
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=1000,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(newsgroups_samples)

lda = LatentDirichletAllocation(n_components=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 10)

### Zadanie

1. Zrób pipeline z powyższego przetwarzania.
1. Spróbuj użyć TF-IDF.
1. Zmień liczbę tematów `n_components` i zobacz co się zmieni.

## Non-negative Matrix Factorization

Kolejnym algorytmem często używanym do ekstrakcji tematów jest [Non-negative Matrix Factorization](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization). 

![](https://upload.wikimedia.org/wikipedia/commons/f/f9/NMF.png)

Metoda ta używa faktoryzacji do przybliżenia macierzy V jako iloczynu macierzy H i W. W wyniku działania algorytmu macierze H i W mają właściwości klasteryzacyjne danych w macierzy V. Dokładnie W staje się macierzą centroidów a H indykatorem przyporządkowania do klastrów poszczególnych elementów macierzy V.

W praktyce często stosuje się tą metodę jako zamiennik PCA, lecz tylko dla danych pozytywnych, oraz do ekstrakcji tematów.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=1000,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(newsgroups_samples)

nmf = NMF(n_components=10, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 10)

### Zadanie

1. Zmień ilość tematów `n_components`.
1. Spróbuj innej funkcji straty `beta_loss`; zobacz [dokumentację](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html#sklearn.decomposition.NMF).